# Segmenting and clustering neighborhoods in Toronto,Canada

## section 1

### Using beautiful soup to get the data and cleaning it

In [1]:
import pandas as pd
import numpy as np

In [3]:
pip install lxml

     |████████████████████████████████| 5.5MB 8.9MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [4]:
import requests
import lxml.html as lh
from bs4 import BeautifulSoup

In [5]:
website_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

In [6]:
soup = BeautifulSoup(website_url,'lxml')

In [7]:
table = soup.find('table',{'class':'wikitable sortable'})
table_heading = table.find_all("th")
table_heading

[<th>Postal Code
 </th>,
 <th>Borough
 </th>,
 <th>Neighborhood
 </th>]

In [8]:
headings = []

for th in table.find_all('th'):
    headings.append(th.text.strip())

In [9]:
df = pd.DataFrame(columns = headings)
df

,Postal Code,Borough,Neighborhood


In [10]:
table_data = []
for tr_cell in table.find_all("tr"):
    row_data = []
    for td_cell in tr_cell.find_all("td"):
        row_data.append(td_cell.text.strip())
    table_data.append(row_data)

In [11]:
for to_append in table_data:
    if len(to_append) != 0:
        a_series = pd.Series(to_append, index = df.columns)
        df = df.append(a_series, ignore_index=True)

In [12]:
df

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,
1,M2A,Not assigned,
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
...,...,...,...
175,M5Z,Not assigned,
176,M6Z,Not assigned,
177,M7Z,Not assigned,
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


In [13]:
df = df[df['Borough'] != 'Not assigned'].reset_index()

In [14]:
df.head()

,index,Postal Code,Borough,Neighborhood
0,2,M3A,North York,Parkwoods
1,3,M4A,North York,Victoria Village
2,4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,5,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [15]:
df1 = df.groupby(['Postal Code', 'Borough'], sort=False).agg( ', '.join)

In [16]:
df1.reset_index(inplace=True)

In [17]:
df1['Neighborhood'] = np.where(df1['Neighborhood'] == 'Not assigned',df1['Borough'], df1['Neighborhood'])

In [18]:
df1.head()

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [19]:
df1.shape

(103, 3)

## section 2

### Adding Coordinates to postal code

In [23]:
postal_code = pd.read_csv('http://cocl.us/Geospatial_data')
postal_code

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


In [24]:
postal_code[postal_code['Postal Code'] == 'M8Z']

,Postal Code,Latitude,Longitude
92,M8Z,43.628841,-79.520999


In [25]:
full_table = pd.merge(df1, postal_code, on='Postal Code')

In [26]:
full_table.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


## section 3

In [27]:
toronto = full_table[full_table['Borough'].str.contains('Toronto',regex=False)]
toronto

,Postal Code,Borough,Neighborhood,Latitude,Longitude
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
19,M4E,East Toronto,The Beaches,43.676357,-79.293031
20,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
24,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
25,M6G,Downtown Toronto,Christie,43.669542,-79.422564
30,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568
31,M6H,West Toronto,"Dufferin, Dovercourt Village",43.669005,-79.442259


In [28]:
import folium # plotting library
map_toronto = folium.Map(location=[43.651070,-79.347015],zoom_start=10)

for lat,lng,borough,neighborhood in zip(toronto['Latitude'],toronto['Longitude'],toronto['Borough'],toronto['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
    [lat,lng],
    radius=5,
    popup=label,
    color='blue',
    fill=True,
    fill_color='#3186cc',
    fill_opacity=0.7,
    parse_html=False).add_to(map_toronto)
map_toronto

In [29]:
CLIENT_ID = 'VEZTE0WLFZPUP01YMZWMUCULBOX0RRPRQGFDMD2PYXYWJRNC'      
CLIENT_SECRET = 'GYLM3IBRBHDFXA5RTNBR4PUYBRMCVX5LOMBSN1B2POAARBPN'   
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: VEZTE0WLFZPUP01YMZWMUCULBOX0RRPRQGFDMD2PYXYWJRNC
CLIENT_SECRET:GYLM3IBRBHDFXA5RTNBR4PUYBRMCVX5LOMBSN1B2POAARBPN


In [30]:
def NearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [31]:
radius = 500
LIMIT = 100 # this will retreive data for only 100 venues per neighbourhood
Toronto_venues = NearbyVenues(names = toronto['Neighborhood'], latitudes = toronto['Latitude'], longitudes = toronto['Longitude'])

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West, Forest Hill Road Park
High Park, The Junction South
North Toronto West,  Lawrence Park
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport


In [32]:
print(Toronto_venues.shape)
Toronto_venues.head(10)

(1612, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot
3,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
4,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
5,"Regent Park, Harbourfront",43.65426,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant
6,"Regent Park, Harbourfront",43.65426,-79.360636,Corktown Common,43.655618,-79.356211,Park
7,"Regent Park, Harbourfront",43.65426,-79.360636,The Extension Room,43.653313,-79.359725,Gym / Fitness Center
8,"Regent Park, Harbourfront",43.65426,-79.360636,The Distillery Historic District,43.650244,-79.359323,Historic Site
9,"Regent Park, Harbourfront",43.65426,-79.360636,Figs Breakfast & Lunch,43.655675,-79.364503,Breakfast Spot


 So we can see that there are total 1616 venues in the neighbourhoods in our dataset. The dataframe generated gives interesting information about Venue names like Starbucks or Pantheon and their corresponding categories like Coffee Shop or Greek Restaurant.

We can use this interesting information to categorize these neigborhoods using k_Means clustering to get insights about business or real estate investments.

Lets see number of venues returned per neighbourhood

In [33]:
Toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,57,57,57,57,57,57
"Brockton, Parkdale Village, Exhibition Place",23,23,23,23,23,23
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",17,17,17,17,17,17
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",15,15,15,15,15,15
Central Bay Street,64,64,64,64,64,64
Christie,17,17,17,17,17,17
Church and Wellesley,77,77,77,77,77,77
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
Davisville,36,36,36,36,36,36


## Venue categories and their densities

Lets now see how many unique venue categories are there and how rare or frequent is their occurence.

In [34]:
Toronto_venues.groupby('Venue Category').count().sort_values(by='Venue', ascending=False)

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude
Venue Category,,,,,,
Coffee Shop,142,142,142,142,142,142
Café,89,89,89,89,89,89
Restaurant,57,57,57,57,57,57
Italian Restaurant,42,42,42,42,42,42
Hotel,34,34,34,34,34,34
...,...,...,...,...,...,...
College Auditorium,1,1,1,1,1,1
Historic Site,1,1,1,1,1,1
College Cafeteria,1,1,1,1,1,1


We can see from above that Coffe Shop, Cafe, Restaurant , Italian, Restaurant, Park are the most frequently observed venues. The occurence of any of these venues in any neighbourhood will define the category of that neighborhood. Hence we will cluster the neighborhoods on the basis of the frequency of the venue categories.

Lets find out how many unique categories we have.

In [35]:
print('There are {} uniques categories.'.format(len(Toronto_venues['Venue Category'].unique())))

There are 240 uniques categories.
